In [ ]:
%pip install -e ..[dev]

# 🤖 Foundational Models Cloud inference 

In [ ]:
from pprint import pprint
import os
from focoos import DeploymentMode, Focoos, FocoosEnvHostUrl
from dotenv import load_dotenv


load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=FocoosEnvHostUrl.LOCAL)

model = focoos.get_model("focoos_ade_small")
## Only admin can deploy foundational models
# model.deploy(deployment_mode=DeploymentMode.REMOTE, wait=True)

detections = model.remote_infer("./assets/ade_val_034.jpg", threshold=0.4)
pprint(detections)

### Annotated preview


In [ ]:
import supervision as sv
import cv2
import numpy as np
import time

preview = model.infer_preview("./assets/ade_val_034.jpg", threshold=0.6)
preview = cv2.imdecode(np.frombuffer(preview, np.uint8), cv2.IMREAD_COLOR)
sv.plot_image(preview, size=(6, 6))

### Unload model from cloud endpoint

In [ ]:
## Only admin can deploy foundational models
model.unload()

# User Models

### List User Models

In [ ]:
from pprint import pprint
import os
from focoos import Focoos, FocoosEnvHostUrl
from dotenv import load_dotenv


load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=FocoosEnvHostUrl.LOCAL)

models = focoos.list_models()
pprint(models)

### New Model

In [ ]:
from pprint import pprint
import os
from focoos import DeploymentMode, Focoos, FocoosEnvHostUrl
from dotenv import load_dotenv

from focoos.ports import NewModel


load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=FocoosEnvHostUrl.LOCAL)

model = focoos.new_model(
    NewModel(
        name="test-model", focoos_model="focoos_object365", description="Test model"
    )
)
### Get Model Info

In [ ]:
focoos.list_models()

## Deploy user model

In [ ]:
from pprint import pprint
import os
from focoos import Focoos, FocoosEnvHostUrl
from dotenv import load_dotenv


load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=FocoosEnvHostUrl.LOCAL)

model = focoos.get_model("9626a2e410864148")
model_info = model.get_info()
pprint(model_info.location)
model.deploy(deployment_mode=DeploymentMode.REMOTE, wait=True)

In [ ]:
import supervision as sv
import cv2
import numpy as np
import time

preview = model.infer_preview("./assets/ade_val_034.jpg", threshold=0.2)
preview = cv2.imdecode(np.frombuffer(preview, np.uint8), cv2.IMREAD_COLOR)
sv.plot_image(preview, size=(6, 6))

## Train a Model

### List Public Datasets


In [ ]:
from pprint import pprint
import os
from focoos import Focoos, FocoosEnvHostUrl, NewModel
from dotenv import load_dotenv


load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=FocoosEnvHostUrl.LOCAL)

models = focoos.list_models()
pprint(models)

model = focoos.new_model(
    NewModel(
        name="test-model", focoos_model="focoos_object365", description="Test model"
    )
)


model = focoos.get_model("fa94df6806c84c11")

datasets = focoos.list_shared_datasets()
pprint(datasets)

In [ ]:
from focoos.ports import Hyperparameters, NewTrain, TrainInstance


res = model.train(
    NewTrain(
        dataset_ref="11e80dd77806450f",
        instance_type=TrainInstance.ML_G4DN_XLARGE,
        volume_size=50,
        max_runtime_in_seconds=36000,
        hyperparameters=Hyperparameters(
            learning_rate=0.0001, batch_size=16, max_iters=1500
        ),
    )
)
pprint(res)

In [ ]:
completed_status = ["Completed", "Failed"]
import time
from focoos.utils.logger import get_logger

logger = get_logger(__name__)

status = model.train_status()
while status["main_status"] not in completed_status:
    status = model.train_status()
    logger.info(f"Training status: {status['main_status']}")
    pprint(f"Training progress: {status['status_transitions']}")
    time.sleep(30)

In [ ]:
logs = model.train_logs()
pprint(logs)

In [ ]:
from focoos.ports import DeploymentMode
import cv2
import numpy as np
import supervision as sv


model.deploy(deployment_mode=DeploymentMode.REMOTE, wait=True)
res = model.infer_preview("./assets/aquarium.jpg", threshold=0.2)
preview = cv2.imdecode(np.frombuffer(res, np.uint8), cv2.IMREAD_COLOR)
sv.plot_image(preview, size=(6, 6))

### Download Model

In [ ]:
from focoos import Focoos, FocoosEnvHostUrl, DeploymentMode
from dotenv import load_dotenv
import os
from pprint import pprint

focoos = Focoos(
    api_key=os.getenv("FOCOOS_API_KEY"),
    host_url=FocoosEnvHostUrl.LOCAL,
)

model_ref = "focoos_rtdetr"
model_info = focoos.get_model_info(model_ref)

model = focoos.get_model(model_ref)
model.deploy(deployment_mode=DeploymentMode.LOCAL)

latency = model.benchmark(iterations=100, size=640)
print(latency)
